In [1]:
from keras.layers import Input, Embedding, LSTM, Dense, Layer, CNN, RNN, Activation
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.optimizers import Adam

import pandas as pd


Using TensorFlow backend.


In [ ]:
area = ["centre","north","west","south","east"]
food = ["afghan","african","afternoon tea","asian oriental","australasian","australian","austrian","barbeque","basque","belgian","bistro","brazilian","british","canapes","cantonese","caribbean","catalan","chinese","christmas","corsica","creative","crossover","cuban","danish","eastern european","english","eritrean","european","french","fusion","gastropub","german","greek","halal","hungarian","indian","indonesian","international","irish","italian","jamaican","japanese","korean","kosher","latin american","lebanese","light bites","malaysian","mediterranean","mexican","middle eastern","modern american","modern eclectic","modern european","modern global","molecular gastronomy","moroccan","new zealand","north african","north american","north indian","northern european","panasian","persian","polish","polynesian","portuguese","romanian","russian","scandinavian","scottish","seafood","singaporean","south african","south indian","spanish","sri lankan","steakhouse","swedish","swiss","thai","the americas","traditional","turkish","tuscan","unusual","vegetarian","venetian","vietnamese","welsh","world"]
pricerange = ["cheap","moderate","expensive"]

## Approach 1 - Data Base Function

In [ ]:
db = pd.read_json('Camrest.json')

def db_query(belief_vector, tokenizer):
    # parse belief vector and select max
    # how to get tensor index of argmax value?
    # area_max = argmax(belief_vector[:len(area)]) 
    
    # filter df rows
    results = df.loc[df['area']==area and df['pricerange']==pricerange and df['food']==food]
    rest_name = results[0]['name']
    rest_addr = results[0]['address']
    rest_phone = results[0]['phone']
    rest_postal = results[0]['postal']
    
    # tokenize output
    
    # return tokenized vector
    return [name, rest_name, addr, rest_addr, phone, rest_phone, postal, rest_postal]

In [ ]:
## REFERENCES
# https://towardsdatascience.com/building-an-ai-chat-bot-e3a05aa3e75f
# https://github.com/priya-dwivedi/Deep-Learning/blob/master/qa_chat_bot/memory_network_q4.py

max_words = 20
vocab_size = 600

## 1) Data input
inputs = Input(shape=(max_words,))

## 2) Intent
seq = LSTM(max_words)(inputs) # return sequences?

## 3) Belief State
belief = Conv1D(max_words, 1, activation='relu')(inputs)
belief = Conv1D(max_words*2, 1, activation='relu')(belief)
belief = Conv1D(max_words*4, 1, activation='relu')(belief)
belief = MaxPoolingID(int(max_words/2))(belief)
belief = RNN(10)(belief) # size? return sequences?
a_belief = Dense(len(area), activation='softmax')(belief)
f_belief = Dense(len(food), activation='softmax')(belief)
p_belief = Dense(len(pricerange), activation='softmax')(belief)

belief = keras.layers.concatenate([a_belief, f_belief, p_belief])

# How to get these to persist?
belief_output = Dense(len(area)+len(food)+len(pricerange), activation='linear')(belief) 

# Need to dynamically call db query

## 4) Database query
db_result = Input(shape=(8,))

## 5) Policy
policy = keras.layers.concatenate([seq, belief, db_result]) 
policy = Dense(64, activation='tanh')(policy)

## 6) Decode response
response = LSTM(32)(policy)
answer = Dropout(0.3)(answer)
answer = Dense(vocab_size, activation='softmax')(answer)

model = Model(inputs=[inputs, db_result], outputs=[response, belief_output])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])



## Approach 2 - Custom Layer

In [ ]:
class DBQuery(Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        self.db = pd.read_json('Camrest.json')
        super(MyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer. --> Can it not have weights?
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(input_shape[1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        super(DBQuery, self).build(input_shape)  

    def call(self, x):
         # parse belief vector and select max
        # how to get tensor index of argmax value?
        # area_max = argmax(x[:len(area)]) 
        
        
        results = self.df.loc[df['area']==area and df['pricerange']==pricerange and df['food']==food]
        rest_name = results[0]['name']
        rest_addr = results[0]['address']
        rest_phone = results[0]['phone']
        rest_postal = results[0]['postal']
        
        # How to access external tokenizer?
        # How to convert vector to a tensor object?
        vector = [name, rest_name, addr, rest_addr, phone, rest_phone, postal, rest_postal]
        
        return vector

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

In [ ]:
max_words = 20
vocab_size = 600

## 1) Data input
inputs = Input(shape=(max_words,))

## 2) Intent
seq = LSTM(max_words)(inputs) # return sequences?

## 3) Belief State
belief = Conv1D(max_words, 1, activation='relu')(inputs)
belief = Conv1D(max_words*2, 1, activation='relu')(belief)
belief = Conv1D(max_words*4, 1, activation='relu')(belief)
belief = MaxPoolingID(int(max_words/2))(belief)
belief = RNN(10)(belief) # size? return sequences?
a_belief = Dense(len(area), activation='softmax')(belief)
f_belief = Dense(len(food), activation='softmax')(belief)
p_belief = Dense(len(pricerange), activation='softmax')(belief)

# How to get these to persist?
belief = keras.layers.concatenate([a_belief, f_belief, p_belief])

## 4) Database query
db_result = DBQuery(8)(belief)

## 5) Policy
policy = keras.layers.concatenate([seq, belief, db_result]) 
policy = Dense(64, activation='tanh')(policy)

## 6) Decode response
response = LSTM(32)(policy)
answer = Dropout(0.3)(answer)
answer = Dense(vocab_size, activation='softmax')(answer)

model = Model(inputs=[inputs, db_result], outputs=[response, belief_output])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

